In [2]:
import os
import time
import math
import re
import struct
import traceback
import codecs
import posixpath

import bitstring

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from PIL import Image
import IPython.display as display

from JIS0208 import JIS0208

In [3]:
part_filepaths = [
    'ETL9G/ETL9G_01',
    'ETL9G/ETL9G_02',
    'ETL9G/ETL9G_03',
    'ETL9G/ETL9G_04',
    'ETL9G/ETL9G_05',
    'ETL9G/ETL9G_06',
    'ETL9G/ETL9G_07',
    'ETL9G/ETL9G_08',
    'ETL9G/ETL9G_09',
    'ETL9G/ETL9G_10',
    'ETL9G/ETL9G_11',
    'ETL9G/ETL9G_12',
    'ETL9G/ETL9G_13',
    'ETL9G/ETL9G_14',
    'ETL9G/ETL9G_15',
    'ETL9G/ETL9G_16',
    'ETL9G/ETL9G_17',
    'ETL9G/ETL9G_18',
    'ETL9G/ETL9G_19',
    'ETL9G/ETL9G_20',
    'ETL9G/ETL9G_21',
    'ETL9G/ETL9G_22',
    'ETL9G/ETL9G_23',
    'ETL9G/ETL9G_24',
    'ETL9G/ETL9G_25',
    'ETL9G/ETL9G_26',
    'ETL9G/ETL9G_27',
    'ETL9G/ETL9G_28',
    'ETL9G/ETL9G_29',
    'ETL9G/ETL9G_30',
    'ETL9G/ETL9G_31',
    'ETL9G/ETL9G_32',
    'ETL9G/ETL9G_33',
    'ETL9G/ETL9G_34',
    'ETL9G/ETL9G_35',
    'ETL9G/ETL9G_36',
    'ETL9G/ETL9G_37',
    'ETL9G/ETL9G_38',
    'ETL9G/ETL9G_39',
    'ETL9G/ETL9G_40',
    'ETL9G/ETL9G_41',
    'ETL9G/ETL9G_42',
    'ETL9G/ETL9G_43',
    'ETL9G/ETL9G_44',
    'ETL9G/ETL9G_45',
    'ETL9G/ETL9G_46',
    'ETL9G/ETL9G_47',
    'ETL9G/ETL9G_48',
    'ETL9G/ETL9G_49',
    'ETL9G/ETL9G_50',
]

# the code below is for Restart and Run All people (include me)
for part_filepath in part_filepaths:
    if not os.path.exists(part_filepath):
        raise Exception(part_filepath, 'does not exist!')

Exception: ('ETL9G/ETL9G_01', 'does not exist!')

In [ ]:
filename = file_list[0]
print(filename)
file_stream = bitstring.ConstBitStream(filename=filename)

In [ ]:
record = file_stream.readlist(','.join([
    'int:16', # Serial Sheet Number - [0]
    'uint:16', # JIS Character Code (JIS X 0208) - [1]
    '8*uint:8', # JIS Typical Reading (e.g. 'AI.MEDER') - [2:10]
    'int:32', # Serial Data Number - [10]
    '4*int:8', # - [11:15]
    '4*uint:16', # - [15:19]
    '2*int:8', # x and y - [19:21]
    'pad:272',
    'bytes:8128', # 16-graylevel (4-bit) image of 128 x 127 = 16256 pixels
    'pad:56',
]))

record_dict = {
    'JIS_CODE': record[1],
    'image_data': record[-1]
}

print(hex(record_dict['JIS_CODE']))

width = 128
height = 127
img = np.array(Image.frombytes('F', (width, height), record_dict['image_data'], 'bit', 4))
plt.imshow(img)
plt.colorbar()

In [ ]:
print(type(record), len(record))
for idx, v in enumerate(record[:-1]):
    print(f'{idx} - {v}')

In [ ]:
total_samples = 0
record_count = {}

for filename in file_list:
    file_stream = bitstring.ConstBitStream(filename=filename)

    while True:
        try:
            record = file_stream.readlist(','.join([
                'int:16', # Serial Sheet Number - [0]
                'uint:16', # JIS Character Code (JIS X 0208) - [1]
                '8*uint:8', # JIS Typical Reading (e.g. 'AI.MEDER') - [2:10]
                'int:32', # Serial Data Number - [10]
                '4*int:8', # - [11:15]
                '4*uint:16', # - [15:19]
                '2*int:8', # x and y - [19:21]
                'pad:272',
                'bytes:8128', # 16-graylevel (4-bit) image of 128 x 127 = 16256 pixels
                'pad:56',
            ]))
        except:
            break
        
        total_samples += 1
        record_dict = {
            'JIS_CODE': record[1],
            'image_data': record[-1],
        }
        
        jis_code = record_dict['JIS_CODE']
        
        if jis_code in JIS0208.keys():
            unicode_char = JIS0208[jis_code]
        else:
            unicode_char = 'JIS_CODE=' + repr(jis_code)

        if unicode_char in record_count.keys():
            record_count[unicode_char] += 1
        else:
            record_count[unicode_char] = 1

In [ ]:
total_samples

In [ ]:
len(record_count.keys())

In [ ]:
record_count

In [ ]:
csv_filename = 'classes.tsv'
with open(csv_filename, mode='w', encoding='utf-8') as f:
    f.write('class\tnum_samples\n')
    for k in record_count.keys():
        log_str = f'{k}\t{record_count[k]}\n'
        f.write(log_str)
# sort classes
pd_df = pd.read_csv(csv_filename, encoding='utf-8', sep='\t')
pd_df = pd_df.sort_values(['class'])
pd_df.to_csv(csv_filename, encoding='utf-8', index=False, sep='\t')

## Extract to images

In [ ]:
from tqdm import tqdm

In [ ]:
filename

In [ ]:
file_size = os.path.getsize(filename)
file_size

In [ ]:
record_length = 8199 # bytes (8 bits per byte)

In [ ]:
file_size / record_length

In [ ]:
total_samples = 0
record_count = {}

record_length = 8199 # bytes (8 bits per byte)

start_time = time.time()

for file_idx, filename in enumerate(file_list):
    print(f'{file_idx+1}/{len(file_list)}: Loading {filename}...')
    
    file_size = os.path.getsize(filename)
    num_samples = int(file_size / record_length)

    file_stream = bitstring.ConstBitStream(filename=filename)

    for _ in tqdm(range(num_samples)):
        record = file_stream.readlist(','.join([
            'int:16', # Serial Sheet Number - [0]
            'uint:16', # JIS Character Code (JIS X 0208) - [1]
            '8*uint:8', # JIS Typical Reading (e.g. 'AI.MEDER') - [2:10]
            'int:32', # Serial Data Number - [10]
            '4*int:8', # - [11:15]
            '4*uint:16', # - [15:19]
            '2*int:8', # x and y - [19:21]
            'pad:272',
            'bytes:8128', # 16-graylevel (4-bit) image of 128 x 127 = 16256 pixels
            'pad:56',
        ]))
        
        total_samples += 1
        record_dict = {
            'JIS_CODE': record[1],
            'image_data': record[-1],
        }
        
        jis_code = record_dict['JIS_CODE']
        
        if jis_code in JIS0208:
            # get the unicode hex string
            unicode_char = JIS0208[jis_code]
        else:
            unicode_char = jis_code

        if unicode_char in record_count.keys():
            record_count[unicode_char] += 1
        else:
            record_count[unicode_char] = 1
            
        save_dir = 'ETL9G_IMG'
        img_filename = f'{unicode_char}_{str(record_count[unicode_char]).zfill(3)}.png'
        img_dir = f'{save_dir}/{unicode_char}'
        if not os.path.exists(img_dir):
            os.makedirs(img_dir)
        img_path = f'{img_dir}/{img_filename}'
        
        if os.path.exists(img_path):
            continue

        width = 128
        height = 127
        np_img = np.array(Image.frombytes('F', (width, height), record_dict['image_data'], 'bit', 4))
        scaled_img = (np_img * 17).astype(np.uint8)
        pil_img = Image.fromarray(scaled_img)
        pil_img.save(img_path)
    
    total_time = time.time() - start_time
    avg_time = total_time / (file_idx + 1)
    remain_time = (len(file_list) - file_idx - 1) * avg_time
    
    print(f'total_time: {total_time:.2f}s, avg_time: {avg_time:.2f}s, remain_time: {remain_time:.2f}s')

In [ ]:
img_path

In [ ]:
np_img = np.array(Image.frombytes('F', (width, height), record_dict['image_data'], 'bit', 4))
plt.imshow(np_img, cmap='gray')
plt.colorbar()

In [ ]:
scaled_img = (np_img * 17).astype(np.uint8)
plt.imshow(scaled_img, cmap='gray')
plt.colorbar()

In [ ]:
pil_img = Image.fromarray(scaled_img)
pil_img.save(img_path)
img_path